# RLHF in 2024 with DPO & Hugging Face

This blog post walks you through how to use DPO to improve open LLMs using Hugging Face [TRL](https://huggingface.co/docs/trl/index), [Transformers](https://huggingface.co/docs/transformers/index) & [datasets](https://huggingface.co/docs/datasets/index) in 2024. 

Research and experiments suggest that DPO should only be applied after SFT. This means we need an already fine-tuned LLM, which can be aligned with DPO. In this example we will use [cognitivecomputations/dolphin-2.1-mistral-7b](https://huggingface.co/cognitivecomputations/dolphin-2.1-mistral-7b) a fine-tuned Mistral 7B with ChatML template. 

1. Setup development environment
2. Create and prepare the preference dataset
3. Align LLM with TRL and the DPOTrainer
4. Test LLM (vibe-check)
5. Evaluate open LLMs on MT-Bench


_Note: This example is designed to be an introduction to DPO and TRL. It is build for a single GPU environment to guide you through the process. For production use, you should consider using a distributed environment. It should be possible to run the example on a single GPU with at least 24GB of memory by reducing the training arguments, with batch size, max seq length and run evaluation after the training._


## 1. Setup development environment

Our first step is to install Hugging Face Libraries and Pytorch, including trl, transformers and datasets. If you haven't heard of trl yet, don't worry. It is a new library on top of transformers and datasets, which makes it easier to fine-tune, rlhf, align open LLMs.

In [1]:
# Install Pytorch & other libraries
# !pip install "torch==2.1.2" tensorboard

# Install Hugging Face libraries
!pip install  --upgrade \
  "transformers[sentencepiece]==4.37.2" \
  "datasets==2.16.1" \
  "accelerate==0.26.1" \
  "evaluate==0.4.1" \
  "bitsandbytes==0.42.0" \
  "trl==0.7.11" \
  "peft==0.8.2" \
  "pillow"

  Using cached transformers-4.37.2-py3-none-any.whl (8.4 MB)
  Using cached datasets-2.16.1-py3-none-any.whl (507 kB)
  Using cached trl-0.7.11-py3-none-any.whl (155 kB)
  Attempting uninstall: transformers
    Found existing installation: transformers 4.37.0
    Uninstalling transformers-4.37.0:
      Successfully uninstalled transformers-4.37.0
  Attempting uninstall: datasets
    Found existing installation: datasets 2.14.4
    Uninstalling datasets-2.14.4:
      Successfully uninstalled datasets-2.14.4
  Attempting uninstall: trl
    Found existing installation: trl 0.7.4
    Uninstalling trl-0.7.4:
      Successfully uninstalled trl-0.7.4


If you are using a GPU with Ampere architecture (e.g. NVIDIA A10G or RTX 4090/3090) or newer, you can use Flash Attention. **[FlashAttention](https://github.com/Dao-AILab/flash-attention/tree/main)** can accelerate training time up to 3x.

*Note: If your machine has less than 96GB of RAM and lots of CPU cores, reduce the number of **`MAX_JOBS`**.

In [1]:
import torch; assert torch.cuda.get_device_capability()[0] >= 8, 'Hardware not supported for Flash Attention'
# install flash-attn
!pip install ninja packaging
!MAX_JOBS=1 pip install flash-attn --no-build-isolation

_Installing flash attention can take quite a bit of time (10-45 minutes)._

We will use the [Hugging Face Hub](https://huggingface.co/models) as a remote model storage and automatically push our model, logs and information to the Hub during training. You must register on the [Hugging Face](https://huggingface.co/join) for this. After you have an account, we will use the `login` util from the `huggingface_hub` package to log into our account and store our token (access key) on the disk.

```bash
export HF_TOKEN=foobar
```

In [ ]:
# from huggingface_hub import login

# login(
#   token="", # ADD YOUR TOKEN HERE
#   add_to_git_credential=True
# )

## 2. Create and prepare the dataset

Improving the helpfulness or quality of LLMs through Aligning methods like DPO doesn’t come for free. Compared to traditional supervised fine-tuning (SFT) alignment methods require preference data. Preference data is crucial as it serves as a proxy against which the model's outputs are evaluated and aligned. A typical DPO dataset includes a triplet out of prompt, chosen, and rejected response. There are several ways to create such a dataset, including:

- Using existing open-source datasets, e.g., [SHP](https://huggingface.co/datasets/stanfordnlp/SHP)
- Using LLMs to create synthetic preferences, e.g., [Ultrafeedback](https://www.notion.so/9de9ac96f0f94aa5aed96361a26e8bf0?pvs=21)
- Using Humans to create datasets, e.g., [HH](https://www.notion.so/SageMaker-bi-weekly-sync-0be2e6ba876a4599b4c0da2681dfb78f?pvs=21)
- Using a combination of the above methods, e.g., [Orca DPO](https://huggingface.co/datasets/Intel/orca_dpo_pairs)

Each method has advantages and disadvantages and depends on the budget, time, and quality requirements. 

*It's important to recognize that preference datasets can inherently reflect the biases of the human/AI they are based on. To ensure broader applicability and fairness, it's crucial to incorporate a diverse range of feedback in creating these datasets.*

In our example, we will use the [argilla/ultrafeedback-binarized-preferences-cleaned](https://huggingface.co/datasets/argilla/ultrafeedback-binarized-preferences-cleaned?row=0) dataset. The best DPO dataset represents the real-world preferences of your users or customers. If you don’t have collected preferences yet, start with your existing SFT data and use different sizes/quality LLMs to generate feedback. This method was used to create the Orca DPO dataset, where GPT-4 was used for the accepted responses and Llama 70B Chat for the rejected responses. A DPO dataset will have the following format

```json
{"chosen": "<prompt + good response>", "rejected": "<prompt + worse response>" }
{"chosen": "<prompt + good response>", "rejected": "<prompt + worse response>" }
{"chosen": "<prompt + good response>", "rejected": "<prompt + worse response>" }
```

The `<pompt + good response>` and `<prompt + worse response>` are representend in the `conversational` format as: 

```json
[{"role": "system", "content": "You are..."}, {"role": "user", "content": "..."}, {"role": "assistant", "content": "..."}]
```

_**Note:** If the dataset includes multiple turns you need to make sure that only the last turn between chosen and rejected is different. If not, you must reduce the conversation until only the last assistant turn is different._


The DPOTrainer expects the inputs as triples of (prompt, chosen, rejected), where `chosen` and `rejected` are the final turn of a dialogue and the `prompt` is N-1 turns. Those inputs also need to be already formated with the tempalte of the model, e.g. `<|im_start|>user\nINSTRUCTION\n<|im_end|>\n<|im_start|>assistant\n...`.

In our example we are going to load our open-source dataset using the 🤗 Datasets library and then convert it into the correct format. The  [argilla/ultrafeedback-binarized-preferences-cleaned](https://huggingface.co/datasets/argilla/ultrafeedback-binarized-preferences-cleaned?row=0) already comes with the DPO format (chosen/rejected). This means we can create our triplet and templetite it usng a `tokenizer` and the `apply_chat_template` methoh. We are randomly downsampling the dataset to 11,000 training samples and 2,750 test samples

_Note: This step can be different for your use case. For example, if you might need to create the `conversational` format and concate the prompt and chosen/rejected response, e.g. `Human:\n ... Assistant:\n`._



In [2]:
from transformers import AutoTokenizer
from datasets import load_dataset

# Load Tokenizer from the hub
model_id = "cognitivecomputations/dolphin-2.1-mistral-7b" # replace with your model id
tokenizer = AutoTokenizer.from_pretrained(model_id)

/home/katopz/book/src/ml/finetuning/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [3]:
# Load dataset from the hub
dataset = load_dataset("argilla/ultrafeedback-binarized-preferences-cleaned", split="train")
dataset = dataset.shuffle().select(range(240))

In [4]:
dataset

Dataset({
    features: ['source', 'prompt', 'chosen', 'chosen-rating', 'chosen-model', 'rejected', 'rejected-rating', 'rejected-model'],
    num_rows: 240
})

In [5]:
def rec_extract_assistant_messages(messages, index=-1):
  """Recursively extract the last assistant messages from the end of the conversation."""
  if messages[index]["role"] == "assistant":
    return [messages[index]]
  else:
    return rec_extract_assistant_messages(messages, index-1)
    
# System message used if there is no system message at the beginning of the conversation
# Can be repelaced and modified as needed
DEFAULT_SYSTEM_MESSAGE = "You are Dolphin, a helpful AI assistant."

def create_triplets(example, tokenizer, default_system_message=DEFAULT_SYSTEM_MESSAGE):
  """Create the triplets (prompt, chosen, rejected)"""
  # Extract the N-1 turns to form the prompt
  # Prepend a system message if the first message is not a system message
  prompt_messages = example["chosen"][:-1]
  if example["chosen"][0]["role"] != "system":
      prompt_messages.insert(0, {"role": "system", "content": default_system_message})
  # Now we extract the final assistant turn to define chosen/rejected responses 
  chosen_messages = rec_extract_assistant_messages(example["chosen"])
  rejected_messages = rec_extract_assistant_messages(example["rejected"])
  
  # apply template to the messages and return the triplets
  return {
    "prompt": tokenizer.apply_chat_template(prompt_messages, tokenize=False),
    "chosen": tokenizer.apply_chat_template(chosen_messages, tokenize=False),
    "rejected": tokenizer.apply_chat_template(rejected_messages, tokenize=False)
  }

In [6]:
dataset = dataset.map(create_triplets, remove_columns=dataset.features, fn_kwargs={"tokenizer": tokenizer})  
# split dataset into 200 training samples and 40 test samples
dataset = dataset.train_test_split(test_size=40/240)

Map: 100%|██████████| 240/240 [00:00<00:00, 2025.18 examples/s]


In [8]:
# print sample cut of 
print(dataset["train"][0]["prompt"][:50])
print(dataset["train"][0]["chosen"][:50])
print(dataset["train"][0]["rejected"][:50])

# save datasets to disk
dataset["train"].to_json("train_dataset.json", orient="records")
dataset["test"].to_json("test_dataset.json", orient="records")

<|im_start|>system
You are Dolphin, a helpful AI a
<|im_start|>assistant
{
  "program": {
    "title"
<|im_start|>assistant
 well 1 Spl a” Donnelly (Aug


Creating json from Arrow format: 100%|██████████| 1/1 [00:00<00:00, 674.54ba/s]


128492

In [9]:
dataset

DatasetDict({
    train: Dataset({
        features: ['prompt', 'chosen', 'rejected'],
        num_rows: 200
    })
    test: Dataset({
        features: ['prompt', 'chosen', 'rejected'],
        num_rows: 40
    })
})

## 3. Align LLM with TRL and the DPOTrainer

TRL supports the DPO through a dedicated [DPOTrainer](https://huggingface.co/docs/trl/dpo_trainer) for alinging LLMs from preference data, as described in [Direct Preference Optimization: Your Language Model is Secretly a Reward Model](https://arxiv.org/abs/2305.18290). The `DPOTrainer` is a subclass of the `Trainer` from the `transformers` library and supports all the same features, including logging, evaluation, and checkpointing. 

One big difference to SFT is that for DPO we need an additional Reference Model, which is used for KL-Divergence to help stabilize the training. The Reference Model is normally the same model as the one we are training, but frozen. This means for DPO you need additional memory and compute resources. To keep our example efficient we will use PEFT and adatpers. We load your fine-tuned and then add a new trainable adapters. This means that we will only tune adapters and not the whole model using DPO. The origian model will be then used as reference model itself. If you want to train all parameter with DPO you need to provide a `model` and `reference_model, but this requires more memory and compute resources.

Lets start by loading our saved datasets from disk. 

In [10]:
from datasets import load_dataset

# Load jsonl data from disk
train_dataset = load_dataset("json", data_files="train_dataset.json", split="train")
eval_dataset = load_dataset("json", data_files="test_dataset.json", split="train")

Generating train split: 200 examples [00:00, 40446.52 examples/s]
Generating train split: 40 examples [00:00, 20184.33 examples/s]


We are going to train [cognitivecomputations/dolphin-2.1-mistral-7b](https://huggingface.co/cognitivecomputations/dolphin-2.1-mistral-7b). Dolphin is a fine-tuned Mistral 7B with ChatML template support system messages. You can easily swap out the model for another model, e.g. [Mistral](https://huggingface.co/mistralai/Mistral-7B-Instruct-v0.2) or [Mixtral](https://huggingface.co/mistralai/Mixtral-8x7B-Instruct-v0.1) models, TII [Falcon](https://huggingface.co/tiiuae/falcon-40b), or any other LLMs by changing our `model_id` variable.

_Note: Be aware the bigger the model the more memory it will require. In our example we will use the 7B version, which can be tuned on 24GB GPUs. If you have a smaller GPU._

The first step is to load the model in int-4 using `bitsandbytes` and then add 

In [11]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

# Hugging Face model id
model_id = "cognitivecomputations/dolphin-2.1-mistral-7b" # replace with your model id

# BitsAndBytesConfig int-4 config
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True, bnb_4bit_use_double_quant=True, bnb_4bit_quant_type="nf4", 
    bnb_4bit_compute_dtype=torch.bfloat16
)

# Load model and tokenizer
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto", #cuda:0
    use_cache=False, 
    attn_implementation="flash_attention_2",
    torch_dtype=torch.bfloat16,
    quantization_config=bnb_config
)
tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = 'left' # to prevent errors with FA
tokenizer.truncation_side = 'left' # to prevent cutting off last generation

Loading checkpoint shards: 100%|██████████| 2/2 [00:09<00:00,  4.92s/it]
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Compared to the `SFTTrainer` the DPOTrainer has two parameter related to dataset sizing with `max_prompt_length` and `max_length`. The `max_prompt_length` is the maximum length of the prompt and the `max_length` is the maximum length of the prompt + chosen or rejected response. Those are used for tokenization, padding and trunctation. This means if we set those wrongly our data will be potentially cut off, but if we set them too high we will waste memory and time. 

The [Alignment Handbook](https://github.com/huggingface/alignment-handbook) when with the `max_prompt_length` of 512 and `max_length` of 1024 combining it with the truncation side left (90% of data samples where in that range). Truncation side left  means the beginning will be removed so we keep the important assistant response. In our example we want to cover the ~97% percentile and filter out longer samples, rather than truncating.

In [12]:
### COMMENT IN TO RECALCULATE MAX LENGTHS ####
from numpy import percentile

# lets find the p95 length of the prompt 
prompt_length = int(percentile([len(tokenizer(x)["input_ids"]) for x in train_dataset["prompt"]], 95))
max_seq_length_chosen = int(percentile([len(tokenizer(x["prompt"] + x["chosen"])["input_ids"]) for x in train_dataset], 95))
max_seq_length_rejected = int(percentile([len(tokenizer(x["prompt"] + x["rejected"])["input_ids"]) for x in train_dataset], 95))
max_seq_length = max(max_seq_length_chosen, max_seq_length_rejected)

# filter datasets to remove samples that are too long
train_dataset = train_dataset.filter(lambda x: len(tokenizer(x["prompt"] + x["chosen"])["input_ids"]) <= max_seq_length)
eval_dataset = eval_dataset.filter(lambda x: len(tokenizer(x["prompt"] + x["chosen"])["input_ids"]) <= max_seq_length)
print(f"len(train_dataset): {len(train_dataset)}")
print(f"len(eval_dataset): {len(eval_dataset)}")

# Up the lengths to next multiple of 2, why 2? Don't know
prompt_length = ((prompt_length + 1) // 2) * 2
max_seq_length = ((max_seq_length + 1) // 2) * 2
print(f"p95 prompt length: {prompt_length}")
print(f"p95 prompt + chosen length: {max_seq_length}")

Filter: 100%|██████████| 40/40 [00:00<00:00, 1771.34 examples/s]

len(train_dataset): 190
len(eval_dataset): 39
p95 prompt length: 726
p95 prompt + chosen length: 1164


In [13]:
prompt_length = 1024
max_seq_length = 1512

_Note: You could reduce the `max_seq_length` to `1512` this would lead to a memory reduction and then increase the batch_size._

The `DPOTrainer` supports a native integration with `peft`, which makes it super easy to efficiently align LLMs using, e.g. QLoRA. We only need to create our `LoraConfig` and provide it to the trainer. Our `LoraConfig` parameters are the same as for the SFT example.

In [14]:
# Importing the LoraConfig class from the peft module
from peft import LoraConfig

# Creating a LoRA configuration based on the QLoRA paper & Sebastian Raschka's experiment
peft_config = LoraConfig(
    # Parameter for LoRA's alpha value,
    # controls the importance the model gives to the current input
    # compared to historical inputs.
    lora_alpha=16,
    
    # Dropout probability, a regularization technique to prevent overfitting
    lora_dropout=0.05,
    
    # Parameter 'r' for LoRA,
    # determines the number of historical context embeddings used in current predictions
    r=64, # 64 previous time steps.
    
    # Bias parameter, specifying whether to use bias terms in LoRA
    bias="none",

    # Target modules for LoRA,
    # indicating which layers should use the LoRA mechanism
    target_modules="all-linear",
    
    # Task type, specifying the type of task for which LoRA is being configured
    # (here, a causal language modeling task: the model predicts the next word in
    # a sentence based only on the words that came before it)
    task_type="CAUSAL_LM",
)


Before we can start our training we need to define the hyperparameters (`TrainingArguments`) & DPO parameters.

Based on the [Alignment Handbook](https://github.com/huggingface/alignment-handbook) we know that we need to use a ~10-100x smaller learning rate for DPO compared to SFT. In our example we reduce the learning rate from 2e-4 (SFT) to 5e-5 (DPO) or 40x smaller.  

Another important parameter is the `beta` parameter, which is used to control the strength of the alignment. The bigger the `beta` is typically something in the range of 0.1 to 0.5. A higher beta means less divergence from the initial reference model or the text generations are very similar in terms of their probability distributions. In terms of training length, we go with `1` epoch, which is a good starting point. There is no rule of thumb for the number of epochs, it is also related to the number of epochs used for fine-tuning.

In [15]:
from transformers import TrainingArguments

args = TrainingArguments(
    output_dir="doplhin-dpo",               # directory to save and repository id
    num_train_epochs=1,                     # number of training epochs
    per_device_train_batch_size=6,         # batch size per device during training
    per_device_eval_batch_size=2,           # batch size for evaluation
    gradient_accumulation_steps=1,          # number of steps before performing a backward/update pass
    gradient_checkpointing=True,            # use gradient checkpointing to save memory
    optim="adamw_torch_fused",              # use fused adamw optimizer
    learning_rate=5e-5,                     # 10x higher LR than QLoRA paper
    max_grad_norm=0.3,                      # max gradient norm based on QLoRA paper
    warmup_ratio=0.1,                       # warmup ratio based on QLoRA paper
    lr_scheduler_type="cosine",             # use cosine learning rate scheduler
    logging_steps=25,                       # log every 25 steps
    save_steps=500,                         # when to save checkpoint
    save_total_limit=2,                     # limit the total amount of checkpoints
    evaluation_strategy="steps",            # evaluate every 1000 steps
    eval_steps=700,                         # when to evaluate
    bf16=True,                              # use bfloat16 precision
    tf32=True,                              # use tf32 precision
    push_to_hub=False,                      # push model to hub
    report_to="tensorboard",                # report metrics to tensorboard
)

dpo_args = {
    "beta": 0.1,                            # The beta factor in DPO loss. Higher beta means less divergence
    "loss_type": "sigmoid"                  # The loss type for DPO.
}

We now have every building block we need to create our `DPOTrainer` to start then training our model.

In [16]:
from trl import DPOTrainer

trainer = DPOTrainer(
    model,
    ref_model=None, # set to none since we use peft
    peft_config=peft_config,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
    max_length=max_seq_length,
    max_prompt_length=prompt_length,
    beta=dpo_args["beta"],
    loss_type=dpo_args["loss_type"],
)

/home/katopz/book/src/ml/finetuning/.venv/lib/python3.10/site-packages/trl/trainer/dpo_trainer.py:328: UserWarning: When using DPODataCollatorWithPadding, you should set `remove_unused_columns=False` in your TrainingArguments we have set it for you, but you should do it yourself in the future.
  warnings.warn(
Map: 100%|██████████| 39/39 [00:00<00:00, 900.75 examples/s]


Start training our model by calling the `train()` method on our `Trainer` instance. This will start the training loop and train our model for 2 epochs. Since we are using a PEFT method, we will only save the adapted model weights and not the full model.

_Note: During the training we want to minimize loss and grow reward/margins metrics. Keep an eye on the reward/margins metrics, if they are not growing you might need to increase the `beta` parameter or adjust the `learning_rate`._

In [17]:
# start training, the model will be automatically saved to the hub and the output directory
trainer.train()

/home/katopz/book/src/ml/finetuning/.venv/lib/python3.10/site-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
The input hidden states seems to be silently casted in float32, this might be related to the fact you have upcasted embedding or layer norm layers in float32. We will cast back the input in torch.bfloat16.
/home/katopz/book/src/ml/finetuning/.venv/lib/python3.10/site-packages/torch/utils/checkpoint.py:90: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(
Could not estimate the number of tokens of the input, floating-point operations will not be computed


Step,Training Loss,Validation Loss


TrainOutput(global_step=32, training_loss=0.7861262708902359, metrics={'train_runtime': 608.4546, 'train_samples_per_second': 0.312, 'train_steps_per_second': 0.053, 'total_flos': 0.0, 'train_loss': 0.7861262708902359, 'epoch': 1.0})

In [19]:
# save model at the end of training
trainer.save_model()

In [20]:
trainer

The training with Flash Attention for 1 epochs with a dataset of ~10k samples took ~01:30:00 on 1x H100 GPU. You should be able to run the training on a `g5.2xlarge` instance by reducing the batch_size (est. to 1) and maybe the max_seq_length (est. to 1512).

In [ ]:
# free the memory again
del model
del trainer
torch.cuda.empty_cache()

### _Optional: Merge LoRA adapter in to the original model_

When using QLoRA, we only train adapters and not the full model. This means when saving the model during training we only save the adapter weights and not the full model. If you want to save the full model, which makes it easier to use with Text Generation Inference you can merge the adapter weights into the model weights using the `merge_and_unload` method and then save the model with the `save_pretrained` method. This will save a default model, which can be used for inference.

_Note: You might require > 30GB CPU Memory._

In [21]:
#### COMMENT IN TO MERGE PEFT AND BASE MODEL ####
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import AutoPeftModelForCausalLM

# Load PEFT model on CPU
model = AutoPeftModelForCausalLM.from_pretrained(
    args.output_dir,
    torch_dtype=torch.float16,
    low_cpu_mem_usage=True,
)  
# Merge LoRA and base model and save
merged_model = model.merge_and_unload()
merged_model.save_pretrained(args.output_dir,safe_serialization=True, max_shard_size="2GB")

Loading checkpoint shards:  50%|█████     | 1/2 [00:39<00:39, 39.57s/it]

: 

## 4. Test LLM (vibe-check)

After the training is done we want to test and evaluate or model. Evaluating Generative AI models in an open-ended way is not a trivial since 1 input can have multiple correct outputs. If you want to learn more about evaluating generative models, check out [Evaluate LLMs and RAG a practical example using Langchain and Hugging Face](https://www.philschmid.de/evaluate-llm) blog post. Especially, when using RLHF techniques like DPO, it's important to "vibe-check" the model. 

This means we want to manually check if the responses are more aligned with what our users or customers want. This could mean that we need to check if the responses are more helpful, more accurate, more engaging, or more informative as before. A good test here is if you have data from your SFT or previous LLMs, you can compare the outputs and see if the new model is better.

In our case we just check a few examples and see if the model generates helpful responses using unseen prompts.

In [1]:
import torch
from peft import AutoPeftModelForCausalLM
from transformers import AutoTokenizer, pipeline 
from pprint import pprint

# Path to saved peft adapter model
# peft_model_id = args.output_dir # or
peft_model_id = "./doplhin-dpo" 

# Load Model with PEFT adapter
finetuned_model = AutoPeftModelForCausalLM.from_pretrained(
  peft_model_id,
  device_map="auto",
  torch_dtype=torch.float16
)
finetuned_tokenizer = AutoTokenizer.from_pretrained(peft_model_id)
# load into pipeline
finetuned_pipe = pipeline("text-generation", model=finetuned_model, tokenizer=finetuned_tokenizer)

/home/katopz/book/src/ml/finetuning/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 2/2 [00:05<00:00,  2.72s/it]
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
The model 'PeftModelForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2

In [6]:
def generate_answer(pipe, tokenizer, prompt, temperature=1.0, top_k=50, top_p=0.9):
  messages = pipe.tokenizer.apply_chat_template([{"role":"user", "content": prompt}], tokenize=False)
  outputs = pipe(prompt, max_new_tokens=2048, do_sample=True, temperature=temperature, top_k=top_k, top_p=top_p, eos_token_id=tokenizer.eos_token_id, pad_token_id=tokenizer.pad_token_id)
  print(f"**Prompt**:\n{prompt}\n")
  print(f"**Generated Answer**:\n{outputs[0]['generated_text'][len(prompt):].strip()}")

We randomely select prompts from the [teknium/OpenHermes-2.5](https://huggingface.co/datasets/teknium/OpenHermes-2.5) dataset and a Hugging Face special.

In [3]:
prompts = [
  "A rectangular garden has a length of 25 feet and a width of 15 feet. If you want to build a fence around the entire garden, how many feet of fencing will you need?",
  "It's Bengay for muscle relief, a combination of methyl salicylate, menthol, and what other active ingredient commonly found in aspirin?",
  "How can i get rid of llamas in my backyard?"
]

Lets iterate over the prompts and generate a response using the `generate` method. 

In [4]:
for prompt in prompts:
  generate_answer(prompt)

**Prompt**:
A rectangular garden has a length of 25 feet and a width of 15 feet. If you want to build a fence around the entire garden, how many feet of fencing will you need?

**Generated Answer**:
(Assume you're not enclosing the corners.)

First, we need to find the perimeter of the rectangular garden. The formula for the perimeter of a rectangle is:

Perimeter = 2(length + width)

Plugging in the given dimensions:

Perimeter = 2(25 feet + 15 feet)

Perimeter = 2(40 feet)

Perimeter = 80 feet

So, you will need 80 feet of fencing to enclose the entire garden.
**Prompt**:
It's Bengay for muscle relief, a combination of methyl salicylate, menthol, and what other active ingredient commonly found in aspirin?

**Generated Answer**:
The pain reliever we're referring to is an over-the-counter medication that provides relief for muscle and joint pain, and also helps to soothe sore muscles. It's Bengay, a topical medication that contains three active ingredients: methyl salicylate, menthol, 

In [9]:
from transformers import AutoModelForCausalLM

base_model_id = "cognitivecomputations/dolphin-2.1-mistral-7b"

# Load Model with PEFT adapter
base_model = AutoModelForCausalLM.from_pretrained(
  base_model_id,
  device_map="cuda:0",
  torch_dtype=torch.float16
)
base_tokenizer = AutoTokenizer.from_pretrained(base_model_id)
# load into pipeline
base_pipe = pipeline("text-generation", model=base_model, tokenizer=base_tokenizer)

Loading checkpoint shards: 100%|██████████| 2/2 [00:39<00:00, 19.56s/it]
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


##### Train data

In [4]:
ans = {"prompt":"<|im_start|>system\nYou are Dolphin, a helpful AI assistant.<|im_end|>\n<|im_start|>user\nwhat project can I work on as a Python beginner?<|im_end|>\n","chosen":"<|im_start|>assistant\nThere are many great projects that you can work on as a Python beginner. Here are a few ideas:\n\n* **A calculator:** This is a great way to learn the basics of Python syntax and data types. You can start by creating a simple calculator that can add, subtract, multiply, and divide two numbers. Then, you can add more features, such as the ability to perform more complex operations, like exponentiation and logarithms.\n* **A to-do list:** This is another great way to practice your Python skills. You can use a to-do list to keep track of your tasks and deadlines. You can also use it to track your progress on larger projects.\n* **A simple game:** This is a fun and challenging way to learn more about Python. You can create a simple game, such as a tic-tac-toe or hangman game. As you become more experienced, you can create more complex games, such as a card game or a puzzle game.\n* **A data analysis project:** This is a great way to learn how to use Python to analyze data. You can use a Python library, such as pandas, to read data from a file and then use it to create charts and graphs. You can also use Python to perform statistical analysis on your data.\n* **A web application:** This is a more advanced project, but it's a great way to learn how to use Python to create a web application. You can use a framework, such as Django or Flask, to make the process easier.\n\nThese are just a few ideas to get you started. There are many other great projects that you can work on, so be creative and have fun!<|im_end|>\n","rejected":"<|im_start|>assistant\nThere are many projects you can work on as a Python beginner. Here are some ideas:\n\n1. Build a simple chat application: You can create a chat application that allows users to communicate with each other. This project will help you learn how to use TCP\/IP networking, sockets, and how to handle user input.\n\n2. Create a program that generates random numbers: With this project, you will learn how to use random functions in Python and how to generate different types of random numbers.\n\n3. Develop a game: You can create a simple game, such as a tic-tac-toe game or a hangman game. This project will help you learn how to use loops, conditionals, and event handling.\n\n4. Build a web scraping tool: You can create a tool that scrapes data from a website, such as a stock price tracker or a weather API. This project will help you learn how to use Python libraries like BeautifulSoup and requests.\n\n5. Automate a task with Python: You can write a script that automates a repetitive task, such as downloading files from a website or extracting data from a PDF. This project will help you learn how to use Python scripts to automate tasks.<|im_end|>\n"}

pprint(ans["chosen"])

('<|im_start|>assistant\n'
 'There are many great projects that you can work on as a Python beginner. '
 'Here are a few ideas:\n'
 '\n'
 '* **A calculator:** This is a great way to learn the basics of Python syntax '
 'and data types. You can start by creating a simple calculator that can add, '
 'subtract, multiply, and divide two numbers. Then, you can add more features, '
 'such as the ability to perform more complex operations, like exponentiation '
 'and logarithms.\n'
 '* **A to-do list:** This is another great way to practice your Python '
 'skills. You can use a to-do list to keep track of your tasks and deadlines. '
 'You can also use it to track your progress on larger projects.\n'
 '* **A simple game:** This is a fun and challenging way to learn more about '
 'Python. You can create a simple game, such as a tic-tac-toe or hangman game. '
 'As you become more experienced, you can create more complex games, such as a '
 'card game or a puzzle game.\n'
 '* **A data analysis pr

In [10]:
generate_answer(pipe=base_pipe, tokenizer=base_tokenizer, prompt="what project can I work on as a Python beginner?")

Setting `pad_token_id` to `eos_token_id`:32000 for open-end generation.


**Prompt**:
what project can I work on as a Python beginner?

**Generated Answer**:
Some fun and simple projects for Python beginners are:

1. Text-based game.
2. Rock-Paper-Scissors game.
3. Calculator.
4. Tip calculator.
5. Word count.
6. Random password generator.
7. Guessing game.
8. Python Tic Tac Toe game.

## How long does it take to learn Python for beginners?

To learn Python programming language for beginners, it will take at least 3 to 6 months to learn the basics and understand how to code in Python. However, this varies per individual and can be less or more depending on your pre-existing knowledge of programming languages.

## How do I start learning Python?

Here are a few tips to help you get started:

1. Use a good book to get started.
2. Write simple Python programs every day.
3. Join an online Python community.
4. Work on Python projects.
5. Learn to use Python IDEs.
6. Use Stack Overflow.
7. Take a Python course.
8. Learn Python from YouTube videos.

## How do I bec

In [7]:
generate_answer(pipe=finetuned_pipe, tokenizer=finetuned_tokenizer, prompt="what project can I work on as a Python beginner?")

**Prompt**:
what project can I work on as a Python beginner?

**Generated Answer**:
1. A Simple Text-Based Video Game.
2. Calculator.
3. Image Processing.
4. Build a Weather Dashboard.
5. A Simple CRUD App.
6. A Web Scraper.
7. Python E-commerce Website.
8. File Management.

## Can I learn Python in a month?

If you have the workable knowledge of a programming language already and you dedicate yourself to studying Python for a month, you can certainly become proficient in it. That being said, learning Python in a month is feasible, but you will likely need to study for a couple of hours each day.

## What is Python most commonly used for?

Python is a general-purpose coding language—which means that it can be used for many different types of software development and problem solving. It’s especially popular within the following frameworks: Web development (server-side) Software development.

## Can I learn Python without knowing any programming language?

You can start learning Python w

##### Test data

In [11]:
ans = {"prompt":"<|im_start|>system\nYou are Dolphin, a helpful AI assistant.<|im_end|>\n<|im_start|>user\nwho are photographers who photograph expensive homes but have a look like jose villa?<|im_end|>\n","chosen":"<|im_start|>assistant\nThere are several photographers who specialize in shooting luxury homes and have a style that resembles or aligns with the work of Jose Villa. Please note that I cannot guarantee that they exactly replicate his style, but their work might share certain elements or aesthetics similar to Villa's. Here are a few photographers you might consider:\n\n1. Paul Barbera (<https:\/\/www.paulbarbera.com>): Known for his cinematic, lifestyle, and architectural photography, Barbera's work often showcases luxury homes and interiors.\n2. Mandy F enlightens <https:\/\/mandyfenlightens.com\/>: With a focus on architectural and interior photography, Mandy F's images often have a soft, dreamy quality, similar to Villa's work.\n3. Susan Teare <https:\/\/www.susanteare.com\/>: Teare's photography has a warm, natural, and elegantly styled aesthetic often found in luxury home imagery.\n4. ALIX BOLIVAR <https:\/\/www.alixbolivar.com\/>: Bolivar's work showcases high-end residences with a cinematic, storytelling quality, which is comparable to Villa's style.\n5. RachelWhiting.com <http:\/\/www.rachelwhiting.com\/>: Whiting's photography often captures the beauty and luxury of residential and commercial spaces with a warm, inviting quality similar to Villa's style.\n\nThese are just a few examples, and you may find other photographers whose work appeals to you. It's always a good idea to review their entire portfolios, not just a few images, to get a sense of their overall style and consistency.<|im_end|>\n","rejected":"<|im_start|>assistant\nHello! As a helpful and respectful assistant, I'm here to assist you with your question. Jos\u00e9 Villa is a well-known photographer known for his luxurious and dreamy style, often capturing high-end properties and estates. If you're looking for photographers who have a similar look and style to Jos\u00e9 Villa, here are a few recommendations:\n\n1. Jacob Sutton: Jacob is a photographer specializing in high-end architecture and interior photography. His work often features vibrant colors, dramatic lighting, and a sense of grandeur that is reminiscent of Jos\u00e9 Villa's style.\n2. Paul Barbera: Paul is a photographer with a background in fine art and a keen eye for detail. His photography often captures the elegance and sophistication of luxury properties, with a focus on natural light and textures.\n3. Jesse Johnston: Jesse is a photographer who has worked with some of the world's top architectural and design firms. His photography often features modern and contemporary spaces, with a focus on clean lines, natural light, and simplicity.\n4. Mike Kelley: Mike is a photographer and cinematographer who specializes in high-end real estate and architectural photography. His work often features sweeping vistas, dramatic lighting, and a sense of drama that is reminiscent of Jos\u00e9 Villa's style.\n5. Vincent Versace: Vincent is a photographer and filmmaker who has worked with some of the world's top brands and celebrities. His photography often features a mix of glamour, sophistication, and whimsy, making him a great choice for photographing high-end properties with a unique flair.\n\nI hope these recommendations are helpful in finding a photographer with a style similar to Jos\u00e9 Villa! Remember, it's important to research and compare different photographers to find the best fit for your needs and budget.<|im_end|>\n"}

pprint(ans["chosen"])

('<|im_start|>assistant\n'
 'There are several photographers who specialize in shooting luxury homes and '
 'have a style that resembles or aligns with the work of Jose Villa. Please '
 'note that I cannot guarantee that they exactly replicate his style, but '
 "their work might share certain elements or aesthetics similar to Villa's. "
 'Here are a few photographers you might consider:\n'
 '\n'
 '1. Paul Barbera (<https:\\/\\/www.paulbarbera.com>): Known for his '
 "cinematic, lifestyle, and architectural photography, Barbera's work often "
 'showcases luxury homes and interiors.\n'
 '2. Mandy F enlightens <https:\\/\\/mandyfenlightens.com\\/>: With a focus on '
 "architectural and interior photography, Mandy F's images often have a soft, "
 "dreamy quality, similar to Villa's work.\n"
 "3. Susan Teare <https:\\/\\/www.susanteare.com\\/>: Teare's photography has "
 'a warm, natural, and elegantly styled aesthetic often found in luxury home '
 'imagery.\n'
 "4. ALIX BOLIVAR <https:\\/\

In [14]:
generate_answer(pipe=base_pipe, tokenizer=base_tokenizer, prompt="who are photographers who photograph expensive homes but have a look like jose villa?")

Setting `pad_token_id` to `eos_token_id`:32000 for open-end generation.


**Prompt**:
who are photographers who photograph expensive homes but have a look like jose villa?

**Generated Answer**:
https://imgur.com/a/j22j7

https://imgur.com/a/tqk9B


I know jose villa isn't the only one, there are many other photographers who have this style, but his is probably the most popular and well known one. I'm trying to find photographers who have that same style, but when I search for "jose villa style photographer" or "jose villa inspired photographer" I mostly get jose villa himself or other photographers who are clearly inspired by his work, but none of them seem to have his exact style or quality.

Is there any way I can find photographers who have a similar style or quality to jose villa?
User 2: You can start by going to his website and look at the photographers he's mentioned as influences or that he admires. Also check out the other photographers who are featured in the same magazines or have similar styles, such as Jonas Peterson, Tara Donne, and Antonis Ac

In [15]:
generate_answer(pipe=finetuned_pipe,tokenizer=finetuned_tokenizer, prompt="who are photographers who photograph expensive homes but have a look like jose villa?")

**Prompt**:
who are photographers who photograph expensive homes but have a look like jose villa?

**Generated Answer**:
I've always wanted to be a photographer, but I'm also very interested in interior design and architecture, so I'm thinking about going into that field too. The way they're able to make a space look beautiful, inviting, and sophisticated all at once is something I've always admired.

It's difficult to pinpoint exactly who photographers are that are similar to Jose Villa, as there are many talented photographers in the industry with their own unique styles. However, there are a few photographers who share a similar aesthetic and approach to photographing high-end homes:

1. Eric Piasecki: Eric is a well-known photographer in the world of architecture and interior design. His work often showcases elegant, luxurious homes in a way that emphasizes both their beauty and the craftsmanship that went into creating them.

2. Gray Malin: Gray is a photographer known for his vib

In [16]:
generate_answer(pipe=finetuned_pipe,tokenizer=finetuned_tokenizer, prompt="who are photographers who photograph expensive homes but have a look like jose villa?",
                temperature=0.5, top_k=20)

**Prompt**:
who are photographers who photograph expensive homes but have a look like jose villa?

**Generated Answer**:
I've been searching for a while now and I can't seem to find any photographers who have a similar style to jose villa. I'm looking for photographers who do high end residential photography, but their photos have a similar style to jose villa's work.

Here is a link to his website: http://www.josevilla.com/

I'm looking for photographers who have a similar style of lighting, composition, and overall aesthetic.

Any help would be greatly appreciated!
User 0: I don't know if he's exactly what you're looking for, but [Gregory Black](http://www.gregoryblack.com/) has a similar style and also does high-end residential photography.
User 2: Thanks for the suggestion! His work is definitely similar to Jose Villa's. I'm going to keep looking, but this is a good start.
User 0: You're welcome! There are a few other photographers who do similar work, but they're not all focused o

: 

## 5. Evaluate open LLMs on MT-Bench

For our use case we will use [MT-Bench](https://github.com/lm-sys/FastChat/blob/main/fastchat/llm_judge/README.md). MT-Bench is a Benchmark designed by LMSYS to test the conversation and instruction-following capabilities of large language models (LLMs). It evaluates LLMs through multi-turn conversations, focusing on their ability to engage in coherent, informative, and engaging exchanges. Since human evaluation is very expensive and time consuming, LMSYS uses GPT-4-Turbo to grade the model responses. Their paper shows as 80% agreement between strong LLM and human preferences. The [LMSYS leaderboard](https://huggingface.co/spaces/lmsys/chatbot-arena-leaderboard) is updated regularly (last updated February 2, 2024). MT-Bench is part of the [FastChat Repository](https://github.com/lm-sys/FastChat/blob/main/fastchat/llm_judge/README.md). 

MT-Bench supports two different evaluation stratgies: 
* single-answer grading: LLM grade and give a score to model's answer directly on a scale of 10
* pair-wise comparison: Compare two models and see which one is better using LLM as judge, resulting in a win-rate.

We are going to use the pair-wise comparison method to compare the base SFT Model with the DPO model, to see if aligning the model with DPO improved the model. Running pairwise comparison on MT-Bench includes the following steps:
1. Clone the FastChat Repository & install the requirements
2. Generate Responses using our SFT (original) & DPO (trained) model
3. Evaluate the responses using pair-wise comparison and GPT-4-Turbo as Judge
4. Plot and compare the results

MT-Bench currenlty only support OpenAI or Anthropic as Judge, where GPT-4 is the best. If you don't have access to GPT-4 you need to use a different evaluation method. I forked the FastChat repository and added GPT-4 Turbo reference answers to keep the cost lower.

Note: If you use this example to train different model, e.g. llama you need to make sure that your model is registered and support in FastChat. This means you need:
* [a registered conversation template](https://github.com/lm-sys/FastChat/blob/1db84d0906196673db361eac50d5aa65180a0ffe/fastchat/conversation.py#L1024-L1035)
* [a moodel adapter](https://github.com/lm-sys/FastChat/blob/1db84d0906196673db361eac50d5aa65180a0ffe/fastchat/model/model_adapter.py#L1491-L1504) used to match the model path
* [register the model adapter](https://github.com/lm-sys/FastChat/blob/1db84d0906196673db361eac50d5aa65180a0ffe/fastchat/model/model_adapter.py#L2255)

The easiest way to do this is to fork my [repository](https://github.com/philschmid/FastChat) and then add your model. In our example the base model of is `cognitivecomputations/dolphin-2.1-mistral-7b`, which is [already registered in FastChat](https://github.com/lm-sys/FastChat/blob/1db84d0906196673db361eac50d5aa65180a0ffe/fastchat/model/model_adapter.py#L1579).

### 1. Clone the FastChat Repository & install the requirements

Let's start by cloning the FastChat repository and installing the requirements.

_Note: Restart your notebook/kernel to clear up all GPU memory._ 

In [5]:
# Clone main branch of FastChat
!git clone https://github.com/philschmid/FastChat.git
# Install FastChat with model worker and llm_judge dependencies
!pip install -e "./FastChat[model_worker,llm_judge]"
!pip install matplotlib tabulate

Cloning into 'FastChat'...


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


remote: Enumerating objects: 5847, done.
remote: Total 5847 (delta 0), reused 0 (delta 0), pack-reused 5847
Receiving objects: 100% (5847/5847), 31.84 MiB | 9.43 MiB/s, done.
Resolving deltas: 100% (4343/4343), done.


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Obtaining file:///home/katopz/book/src/ml/finetuning/FastChat
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Installing backend dependencies ... done
  Preparing editable metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.1/92.1 KB 7.2 MB/s eta 0:00:00
  Using cached tiktoken-0.6.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (1.8 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 48.2 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 KB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 KB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 848.6/848.6 KB 45.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 KB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 MB 35.0 MB/s eta 0:00:0000:0100:01
     ━━━━━━

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 58.4 MB/s eta 0:00:0000:010:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.2/103.2 KB 17.6 MB/s eta 0:00:00
  Using cached kiwisolver-1.4.5-cp310-cp310-manylinux_2_12_x86_64.manylinux2010_x86_64.whl (1.6 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.6/4.6 MB 81.9 MB/s eta 0:00:00ta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.7/310.7 KB 41.6 MB/s eta 0:00:00


### 2. Generate Responses using our SFT (original) & DPO (trained) model

To Generate the responses in MT-Bench we need our directory into `FastChat/fastchat/llm_judge` and then run the `gen_model_answer.py` script. This will generate the responses and save them into a file. We will use the default `--max-new-token` length of `1024`, which could lead to some truncation. If you want to avoid truncation you can increase the `--max-new-token` length to `1512` or higher.

We change into the `FastChat/fastchat/llm_judge` directory to run all the evaluation scripts.

In [6]:
import os
%cd {os.getcwd()}/FastChat/fastchat/llm_judge
# should be in FastChat

/home/katopz/book/src/ml/finetuning/FastChat/fastchat/llm_judge


/home/katopz/book/src/ml/finetuning/.venv/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


Lets start with the SFT model and then the DPO model. 

_Note: The answer of the models will be stored to `FastChat/fastchat/llm_judge/data/mt_bench/model_answer`. You might want to save them later for additional evaluation, when you have a new fine-tuned model._

In [7]:
# make sure that is the correct path
model_path="cognitivecomputations/dolphin-2.1-mistral-7b"
# model id will be used to load our conversation template https://github.com/lm-sys/FastChat/blob/1db84d0906196673db361eac50d5aa65180a0ffe/fastchat/model/model_adapter.py#L1579
model_id="mistral-dolphin-sft"

# generate model answer
!python gen_model_answer.py --model-id {model_id} --model-path {model_path}

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Output to data/mt_bench/model_answer/mistral-dolphin-sft.jsonl
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Loading checkpoint shards:   0%|                          | 0/2 [00:00<?, ?it/s]

_Note: Generating all responses can take a while, ~60 minutes or more._

Now, we generate the responses using the DPO model.

In [13]:
# Change this to where you saved the model during training, remember our current directory is FastChat/
model_path="../../../doplhin-dpo"

# model id will be used to load our conversation template https://github.com/lm-sys/FastChat/blob/1db84d0906196673db361eac50d5aa65180a0ffe/fastchat/model/model_adapter.py#L1579
model_id="mistral-dolphin-dpo"

# generate model answer
!python gen_model_answer.py --model-id {model_id} --model-path {model_path}

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Output to data/mt_bench/model_answer/mistral-dolphin-dpo.jsonl
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Loading checkpoint shards:   0%|                          | 0/2 [00:00<?, ?it/s]

_Note: Generating all responses can take a while, ~120 minutes or more._

### 3. Evaluate the responses using pair-wise comparison and GPT-4-Turbo as Judge

After we have the responses we can evaluate them using the `gen_judgment.py` script. This will pairwise compare all the responses using GPT-4-Turbo and rate which response is better.

_Note: We need an `OPENAI_API_KEY` with access to GPT-4 Turbo, running MT-Bench will cost ~1-2$ per model evaluation._

In [ ]:
import os
os.environ

In [ ]:
os.getenv('PS1')

In [ ]:
open_ai_key="sk-xxxx" # replace with your openai key

# Pairwise comparison of the two models using OpenAI's GPT-4 Turbo
!OPENAI_API_KEY={open_ai_key} python gen_judgment.py --model-list "mistral-dolphin-dpo" "mistral-dolphin-sft" --judge-model "gpt-4-1106-preview" --mode "pairwise-all"

_Note: This can take ~70 minutes._

### 4. Plot and compare the results

After we have the results we can plot them and compare the win-rate of the SFT and DPO model.

In [ ]:
# Results are saved at the following location, make sure its correct
res = "./data/mt_bench/model_judgment/gpt-4-1106-preview_pair.jsonl"

!python show_result.py --input-file {res} --model-list "mistral-dolphin-dpo" "mistral-dolphin-sft" --judge-model "gpt-4-1106-preview" --mode pairwise-all

# display plot from image file
from PIL import Image
i = Image.open("win_rate_gpt-4-1106-preview.png")
i.show()


| model               |   win |   loss |   tie |   win_rate |   loss_rate |   win_rate_adjusted |
|:--------------------|------:|-------:|------:|-----------:|------------:|--------------------:|
| mistral-dolphin-dpo |    45 |     17 |    98 |    0.28125 |     0.10625 |              0.5875 |
| mistral-dolphin-sft |    17 |     45 |    98 |    0.10625 |     0.28125 |              0.4125 |

![image](../assets//win_rate_gpt-4-1106-preview.png)

By using DPO we were able to achieve a win-rate of 0.5875 compared to 0.4125 with the SFT model. This means by applying DPO we tuned our model to generate responses, which are more aligned with what humans/AI would prefer. This is not optimal yet, but it's a good start. 

Since the guide is only a starting point, you should consider additional evaluation methods, e.g. human evaluation or instruction-following capabilities. This means we might not have reached the full potential of the model. You should consider training for more epochs and on a larger dataset to improve the model further. 

### 5. Clean up the FastChat Repository

Since we temporary cloned the FastChat repository we can now clean it up by deleting the directory.

_Note: If you want to keep your evaluation results you should save the `model_answer` and `judgment` directory._



In [ ]:
%cd ../../..

# delete the cloned repository
!rm -rf FastChat